0. 라이브러리 및 데이터 불러오기

1. 데이터 전처리
- 활용 데이터
- 데이터 전처리
- EDA

2. 분석 프로세스
- 파생변수 생성
- Data Scaling
- Clustering
- 분석결과 요약

3. 분석결과 및 군집별 대시보드
- 분석결과 해석
- 군집별 대시보드
- 샐러드볼 지수

4. 업무 활용 방안
- 공실 양극화 및 서울교통공사 적자 해소
- 농산물 가격 변동폭 축소 및 안정적 공급
- 정책 활용
- 고용 및 교육 프로그램 증대

5. 유형별 스마트팜 추천 입지
- 보급형 스마트팜 추천 입지
- 유통형 스마트팜 추천 입지
- 생산형 스마트팜 추천 입지


In [7]:
# 필요 패키지 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


- ***Raw_Data 불러오기***

In [8]:
file_subway_onoff = 'C://Users//82102//TIL//Study//data//서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv'
file_subway_rent = "C://Users//82102//TIL//Study//data//서울교통공사_지하상가임대정보_20210531.csv"
file_smartfarm = 'C://Users//82102//TIL//Study//data//서울교통공사_스마트팜 설치현황_20200930.csv'
file_store = "C://Users//82102//TIL//Study//data//소상공인시장진흥공단_상가(상권)정보_서울_202106.csv"
file_subway = "C://Users//82102//TIL//Study//data//서울교통공사_도시철도역사정보_20210409_1618459845117_52128.csv"

- ***분석 프로세스***
1. 유동량
- 데이터: 서울시 지하철 호선/역/시간대별 승하차 인원정보 --> 상대 유동지수

2. 수급처: 소상공인시장진흥공단 상가정보 
- 반경 500m 내 소상공인 count

3. 공간활용: 서울교통공사 지하상가 임대정보 
- 공실/비공실 데이터 구분
- 면적을 통한 파생변수 생성 

## 데이터 전처리

### 서울도시철도역사정보표준데이터
-  서울교통공사 도시철도역사정보에서 역위경도를 사용하고자 함.

In [10]:
file_subway = pd.read_csv("C://Users//82102//TIL//Study//data//서울교통공사_도시철도역사정보_20210409_1618459845117_52128.csv", encoding = 'euc-kr')
file_subway.head()

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
0,150,서울역,S1101,서울 도시철도 1호선,Seoul Station,首爾站,환승역,S1104+I4107,서울 도시철도 4호선+경의중앙선,37.555753,126.972456,서울교통공사,서울특별시 중구 세종대로 지하 2 (남대문로 5가),02-6110-1331,2021-04-09
1,151,시청,S1101,서울 도시철도 1호선,City Hall,市廳,환승역,S1102,서울 도시철도 2호선,37.564717,126.976751,서울교통공사,서울특별시 중구 세종대로 지하 101 (정동),02-6110-1321,2021-04-09
2,152,종각,S1101,서울 도시철도 1호선,Jonggak,鐘閣,일반역,NaN,NaN,37.570203,126.982948,서울교통공사,서울특별시 종로구 종로 지하 55 (종로1가),02-6110-1311,2021-04-09
3,153,종로3가,S1101,서울 도시철도 1호선,Jongno 3(sam)-ga,鍾路三街,환승역,S1103+S1105,서울 도시철도 3호선+서울 도시철도 5호선,37.571394,126.991411,서울교통공사,서울특별시 종로구 종로 지하 129 (종로3가),02-6110-1301,2021-04-09
4,154,종로5가,S1101,서울 도시철도 1호선,Jongno 5(o)-ga,鐘路五街,일반역,NaN,NaN,37.571007,127.001799,서울교통공사,서울특별시 종로구 종로 지하 216 (종로5가),02-6110-1291,2021-04-09


In [11]:
# 필요 컬럼만 추출
seoul_sub = file_subway[['역사명','역위도','역경도','역사도로명주소']].copy()

# 결측치 확인
# isnull().sum()
seoul_sub.isnull().sum()

역사명        0
역위도        0
역경도        0
역사도로명주소    0
dtype: int64

#### 역명 전처리 함수 생성
- 역사명 뒤에 괄호가 붙는 경우 괄호를 모두 제거해준다
- 역사명 '역'이 붙는 경우 모두 oo역으로 통일한다

In [14]:
def preprocess(data,col):
    data[col] = data[col].replace('\([^)]*\)',"",regex =True)
    data[col] = data[col].str.replace(" ","") # 공백제거 정규식
    data[col] = data[col].str.rstrip("역") # '역' 제거
    data[col] += "역"

In [30]:
# 역사명 통일 시켜주기
preprocess(seoul_sub,'역사명')

In [31]:
seoul_sub

,역사명,역위도,역경도,역사도로명주소
0,서울역,37.555753,126.972456,서울특별시 중구 세종대로 지하 2 (남대문로 5가)
1,시청역,37.564717,126.976751,서울특별시 중구 세종대로 지하 101 (정동)
2,종각역,37.570203,126.982948,서울특별시 종로구 종로 지하 55 (종로1가)
3,종로3가역,37.571394,126.991411,서울특별시 종로구 종로 지하 129 (종로3가)
4,종로5가역,37.571007,127.001799,서울특별시 종로구 종로 지하 216 (종로5가)
...,...,...,...,...
287,송파나루역,37.510120,127.113123,서울특별시 송파구 오금로 165
288,한성백제역,37.516865,127.118185,서울특별시 송파구 위례성대로 51
289,올림픽공원역,37.516088,127.130600,서울특별시 송파구 양재대로 1233
290,둔촌오륜역,37.517975,127.140170,서울특별시 송파구 강동대로 327


#### 동일 역명 및 환승역 처리
- 환승역은 역위도, 경도 값이 호선 수 만큼 나오기 때문에 (중복되는)환승역 위경도의 평균값 사용
- 중복역은 4호선 이수역 == 총신대입구역이 같음

In [32]:
len(seoul_sub['역사명'].unique())

252

In [33]:
seoul_sub[(seoul_sub['역사명']=='이수역')|(seoul_sub['역사명']=='총신대입구역')]

,역사명,역위도,역경도,역사도로명주소
238,이수역,37.486682,126.981845,서울특별시 동작구 사당로 지하310 (사당동)


In [34]:
# 이수역, 총신대입구역 위경도 평균값 내고 이수역으로 통일

이수위경도 = seoul_sub[(seoul_sub['역사명']=='이수역')|(seoul_sub['역사명']=='총신대입구역')].mean()
seoul_sub.loc[238,['역위도','역경도']] = 이수위경도  # data.loc[행,렬] = 인덱싱

In [50]:
# 총신대역 drop

# seoul_sub.drop(118,inplace = True) # inplace = True => 변경된 df가 기존 df를 대체하겠다는 의미
# seoul_sub.reset_index(drop=True,inplace=True)

In [36]:
len(seoul_sub['역사명'].unique())

252

In [41]:
# 중복되는 환승역 확인
transfer = seoul_sub.groupby(['역사명']).count() # groupby --> 역사명이 index로 설정됨
transfer[transfer['역위도']>1].index

Index(['가락시장역', '건대입구역', '고속터미널역', '공덕역', '교대역', '군자역', '까치산역', '노원역', '대림역',
       '동대문역', '동대문역사문화공원역', '동묘앞역', '불광역', '사당역', '삼각지역', '서울역', '석촌역', '시청역',
       '신당역', '신설동역', '약수역', '연신내역', '영등포구청역', '오금역', '올림픽공원역', '왕십리역',
       '을지로3가역', '을지로4가역', '잠실역', '종로3가역', '종합운동장역', '천호역', '청구역', '충무로역',
       '충정로역', '태릉입구역', '합정역'],
      dtype='object', name='역사명')

In [51]:
# 환승역 위, 경도 평균 구하기
seoul_sub = seoul_sub.groupby(['역사명'], as_index = False).mean() # 역사명을 index로 설정하지 않고 연산하겠다

print("서울시내 역의 개수 - ",len(seoul_sub['역사명']))

서울시내 역의 개수 -  252


### 서울시 지하철 호선별 역별 시간대별 승하차 인원 정보
- 서울시 지하철 유동인구에 대한 분석을 진행

In [57]:
subway_onoff = pd.read_csv('C://Users//82102//TIL//Study//data//서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv', encoding = 'euc-kr')
subway_onoff.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202107,1호선,동대문,665,13,13154,2112,8755,6552,13558,...,7234,7,1394,0,0,0,1,0,0,20210803
1,202107,1호선,동묘앞,92,0,3197,1077,3575,4910,5452,...,1955,3,1039,0,0,0,0,0,0,20210803
2,202107,1호선,서울역,654,6,8652,6342,11542,33836,34033,...,5824,29,551,0,0,0,0,0,0,20210803
3,202107,1호선,시청,13,0,1803,4616,2797,19665,6210,...,988,6,84,0,0,0,0,0,0,20210803
4,202107,1호선,신설동,295,14,7940,3120,7834,10994,15927,...,4104,21,437,1,0,0,0,0,0,20210803


#### 컬럼 전처리
- 필요없는 컬럼 제거['작업일자']
- seoul_sub 데이터와 컬럼명 동일하게 설정 [지하철역] --> [역사명]
- 코로나 전후 (18년 ~ 21년 데이터만 사용)

In [61]:
# 필요없는 컬럼제거
subway_onoff.drop(columns = '작업일자', inplace =True)

In [62]:
# 2021 ~ 2018년 6월까지 데이터만 사용
subway_onoff = subway_onoff[subway_onoff['사용월']>= 201806]

In [63]:
# 지하철역 --> 역사명 컬럼명 변경(rename(colums ={'기존col:새로운col'}))
subway_onoff.rename(columns = {'지하철역':'역사명'},inplace =True)
subway_onoff

,사용월,호선명,역사명,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,202107,1호선,동대문,665,13,13154,2112,8755,6552,13558,...,2095,7234,7,1394,0,0,0,1,0,0
1,202107,1호선,동묘앞,92,0,3197,1077,3575,4910,5452,...,827,1955,3,1039,0,0,0,0,0,0
2,202107,1호선,서울역,654,6,8652,6342,11542,33836,34033,...,8520,5824,29,551,0,0,0,0,0,0
3,202107,1호선,시청,13,0,1803,4616,2797,19665,6210,...,3016,988,6,84,0,0,0,0,0,0
4,202107,1호선,신설동,295,14,7940,3120,7834,10994,15927,...,1497,4104,21,437,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,201806,중앙선,원덕,40,1,446,13,739,230,1241,...,22,439,3,219,0,0,0,0,0,0
22744,201806,중앙선,중랑,98,2,6584,538,10636,2793,25378,...,1455,6556,119,1632,0,0,0,0,0,0
22745,201806,중앙선,지평,0,0,0,0,0,0,316,...,0,0,0,0,0,0,0,0,0,0
22746,201806,중앙선,팔당,16,0,423,167,863,804,2023,...,301,375,20,100,0,0,0,0,0,0


In [65]:
# 역별 총 승하차 인원 구하기

subway_onoff.groupby(['역사명','호선명'],as_index =False).sum()


,역사명,호선명,사용월,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,4.19민주묘지,우이신설선,7674347,288,79,49271,10148,111717,38039,276098,...,19594,88031,2996,17553,8,150,2,26,0,0
1,가능,경원선,7674347,15137,242,297304,64061,464793,167769,919784,...,52248,251499,3967,63619,3,58,0,0,0,0
2,가락시장,3호선,7674347,475,14,99382,23688,267137,198161,785757,...,96195,240562,10362,70659,48,1525,0,23,0,0
3,가락시장,8호선,7674347,708,10,98117,85233,188162,235639,370415,...,143104,173248,19913,80953,79,12411,5,202,0,0
4,가산디지털단지,7호선,7674347,25141,127,288768,199954,341435,821738,754262,...,506267,263097,48637,75724,103,327,9,52,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,회룡,경원선,7674347,37969,775,525853,75493,956822,237170,2129175,...,61307,457326,6059,160795,3,75,0,0,0,0
617,회현(남대문시장),4호선,7674347,1576,41,81525,33449,136248,813045,213670,...,197915,159259,20716,45303,158,3290,4,42,0,0
618,효창공원앞,6호선,7674347,231,4,89314,22786,183853,153734,573438,...,81428,177892,8962,55529,74,316,0,13,0,0
619,효창공원앞,경의선,7674347,122,4,23973,8429,67277,77014,166179,...,37264,83817,2733,17157,1,12,0,0,0,0


- preprocess() 함수로 역 전처리

In [68]:
# 역사명 통일시켜주기
preprocess(subway_onoff,'역사명')

In [70]:
# 중복역 확인 및 제거
subway_onoff[(subway_onoff.역사명=='이수역')|(subway_onoff.역사명=='총신대입구역')]

,사용월,호선명,역사명,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
115,202107,4호선,총신대입구역,90,6,6214,938,15402,9305,41258,...,1757,7049,1,78,0,0,0,0,0,0
253,202107,7호선,이수역,60,0,4469,983,8664,9353,24483,...,3164,5424,4,373,0,0,0,0,0,0
722,202106,4호선,총신대입구역,84,2,6356,865,16275,9922,46330,...,2334,10009,1,57,0,0,0,0,0,0
860,202106,7호선,이수역,23,0,4625,1044,8971,9219,27027,...,4545,7306,7,561,0,0,0,0,0,0
1330,202105,4호선,총신대입구역,106,3,5782,792,14964,10251,41243,...,2528,9738,4,119,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21229,201808,7호선,이수역,37,1,5428,1105,10262,9602,29423,...,15690,12100,2896,4118,2,109,0,0,0,0
21688,201807,4호선,총신대입구역,79,2,8127,975,20347,11281,55672,...,13549,20642,1970,5746,1,102,0,0,0,0
21820,201807,7호선,이수역,27,1,5726,1227,10955,10325,31371,...,16588,12518,3209,4356,0,100,0,0,0,0
22276,201806,4호선,총신대입구역,79,2,8341,941,20145,11224,53246,...,13347,21610,1782,5723,1,209,0,0,0,0


In [75]:
# 총신대입구역 => 이수역으로 변경

이수역 = subway_onoff[(subway_onoff.역사명=='이수역')|(subway_onoff.역사명=='총신대입구역')].index

for i in 이수역 :
    subway_onoff.loc[i,'역사명'] = '이수역' 

In [76]:
# 총신대 입구역 삭제된 것 확인
subway_onoff[subway_onoff['역사명']=='총신대입구역']

,사용월,호선명,역사명,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원


In [77]:
len(subway_onoff['역사명'].unique())

520

#### 서울 내 존재하는 역의 유동인구만 추출
- seoul_sub과 subway_onoff merge

In [86]:
# seoul_sub 에 없는 역 확인

print(set(subway_onoff['역사명'].unique())-set(seoul_sub['역사명'].unique())) 

{'안양역', '원인재역', '지제역', '북한산보국문역', '기흥역', '세종대왕릉역', '갈매역', '신길온천역', '계양역', '원흥역', '망우역', '소사역', '의정부역', '대공원역', '간석역', '독산역', '아산역', '평택역', '운서역', '굴봉산역', '배방역', '한대앞역', '양정역', '양주역', '화계역', '한남역', '소요산역', '평내호평역', '인천논현역', '도심역', '매교역', '국수역', '과천역', '주엽역', '회기역', '정왕역', '덕정역', '달월역', '곡산역', '백석역', '안산역', '백양리역', '가오리역', '고색역', '사릉역', '정부과천청사역', '강일역', '봉명역', '송탄역', '구반포역', '원덕역', '월계역', '구로역', '오빈역', '청라국제도시역', '구룡역', '월롱역', '흑석역', '아신역', '압구정로데오역', '문산역', '서정리역', '방학역', '부개역', '운정역', '호구포역', '미사역', '직산역', '일산역', '중랑역', '국회의사당역', '천안역', '부천역', '신논현역', '동두천역', '응봉역', '용문역', '삼송역', '동두천중앙역', '오이도역', '임진강역', '숭의역', '서강대역', '대화역', '산본역', '강매역', '한티역', '풍산역', '용산역', '상갈역', '구성역', '지행역', '덕소역', '신목동역', '중앙역', '하남검단산역', '소래포구역', '증미역', '쌍용역', '도봉역', '인천공항2터미널역', '보산역', '상천역', '영통역', '신포역', '오목천역', '서현역', '파주역', '양원역', '여주역', '도화역', '야목역', '검암역', '이천역', '강촌역', '회룡역', '운길산역', '어천역', '당정역', '경기광주역', '정발산역', '성균관대역', '화정역', '사리역', '가양역', '월곶역', '성환역', '동인천역', '금곡역', '중동역', '별내역', '남춘천역', '인하대역',

In [88]:
# 서울시내 지하철 데이터와 merge
sub_onoff = pd.merge(subway_onoff,seoul_sub, how ='inner', on ='역사명').drop(columns = ['역위도','역경도'])
sub_onoff.head()

,사용월,호선명,역사명,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,202107,1호선,동대문역,665,13,13154,2112,8755,6552,13558,...,2095,7234,7,1394,0,0,0,1,0,0
1,202107,4호선,동대문역,189,28,5495,3261,3168,14636,5147,...,2453,4433,7,36,0,0,0,0,0,0
2,202106,1호선,동대문역,715,14,13235,2131,8936,6979,14776,...,2811,8211,16,1434,1,1,0,0,0,0
3,202106,4호선,동대문역,191,19,5780,3227,3112,14788,5309,...,3614,5147,3,70,0,0,0,0,0,0
4,202105,1호선,동대문역,741,10,12809,1940,8433,7185,13545,...,2852,8224,17,1299,0,0,0,1,0,0


In [96]:
# 연도별, 시간대별 평균 구하기 위해 datetime으로 변환 후 연도만 뽑아오기

sub_group = sub_onoff.copy()
sub_group.insert(1, '년', pd.to_datetime(sub_group['사용월'],format = '%Y%m').dt.year) # 1열에 '년' col 삽입 / year만 추출
sub_group.drop(columns =['사용월'], inplace = True)

# 연도별, 역별, 시간대별 평균
sub_group = round(sub_group.groupby(['년','역사명'], as_index = False).mean()) #소수점 정리
sub_group.head()

,년,역사명,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,07시-08시 하차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,2018,가락시장역,17.0,1.0,2819.0,1388.0,6487.0,5892.0,17390.0,11973.0,...,4317.0,7345.0,664.0,2640.0,4.0,302.0,0.0,8.0,0.0,0.0
1,2018,가산디지털단지역,1008.0,4.0,8476.0,3280.0,9571.0,15241.0,18231.0,70398.0,...,13333.0,8091.0,1232.0,2091.0,4.0,12.0,0.0,2.0,0.0,0.0
2,2018,강남구청역,30.0,0.0,1567.0,1000.0,3512.0,8803.0,7584.0,21480.0,...,7687.0,4673.0,969.0,1605.0,1.0,6.0,0.0,0.0,0.0,0.0
3,2018,강남역,278.0,7.0,16529.0,12296.0,25498.0,57998.0,47360.0,172048.0,...,133226.0,42311.0,24166.0,13128.0,160.0,375.0,51.0,87.0,6.0,20.0
4,2018,강동구청역,9.0,1.0,4534.0,1066.0,11098.0,5885.0,34957.0,13895.0,...,2333.0,9695.0,396.0,3295.0,5.0,22.0,0.0,0.0,0.0,0.0


- sub_group 연도별로 분리(18,19,20,21)

In [103]:
subway_18 = sub_group[sub_group['년']==2018].reset_index(drop=True)
subway_19 = sub_group[sub_group['년']==2019].reset_index(drop=True)
subway_20 = sub_group[sub_group['년']==2020].reset_index(drop=True)
subway_21 = sub_group[sub_group['년']==2021].reset_index(drop=True)

In [105]:
# 연도별 data 구조 살펴보기
subway_18.shape, subway_19.shape, subway_20.shape, subway_21.shape

((252, 50), (252, 50), (252, 50), (252, 50))

### 지하상가 임대정보
- 지하철 역사 내 지하상가에 대한 임대정보를 담은 정보
- 지하철 역사 내 공실과 면적을 파악하고 기대 생산량 도출

In [106]:
subway_rent = pd.read_csv("C://Users//82102//TIL//Study//data//서울교통공사_지하상가임대정보_20210531.csv", encoding = 'euc-kr', usecols = ['상가유형','역명','면적'])
# 필요한 컬럼만 추출 usecols
subway_rent.head()

,상가유형,역명,면적
0,개별(일반),서울(1)역,33.00
1,개별(일반),시청(1)역,29.73
2,개별(일반),시청(1)역,57.60
3,개별(일반),시청(1)역,25.00
4,네트워크,시청(1)역,25.00


####  다른 데이터셋과 용어 통일
- 역명 -> 역사명
- prepocess 함수로 괄호제거, +역 등 전처리
- 총신대입구역 -> 이수역으로 통일

In [110]:
# 역명 확인
subway_rent['역명'].unique()[:10]

array(['서울(1)역', '시청(1)역', '종각역', '종로3가(1)역', '종로5가역', '제기동역', '동묘앞(1)역',
       '을지로입구역', '을지로3가(2)역', '을지로4가(2)역'], dtype=object)

In [112]:
# 함수 사용 역명 전처리
preprocess(subway_rent,'역명')

# 확인
subway_rent['역명'].unique()[:10]

array(['서울역', '시청역', '종각역', '종로3가역', '종로5가역', '제기동역', '동묘앞역', '을지로입구역',
       '을지로3가역', '을지로4가역'], dtype=object)

In [114]:
# 컬럼명 통일
subway_rent.rename(columns = {'역명' : '역사명'}, inplace = True)


,상가유형,역사명,면적
587,공실,총신대입구역,14.80
588,개별(일반),총신대입구역,31.90
589,공실,총신대입구역,32.60
590,개별(일반),총신대입구역,16.30
591,개별(일반),총신대입구역,16.30
592,공실,총신대입구역,16.30
593,개별(일반),총신대입구역,16.30
594,개별(일반),총신대입구역,16.30
595,개별(일반),총신대입구역,16.30
596,개별(일반),총신대입구역,16.30


In [117]:
# 이수역 = 총신대 입구역 통일
subway_rent.loc[subway_rent['역사명'] == '총신대입구역', '역사명'] = '이수역' # 인덱싱 => 특정 행, 열의 값을 변경해줄때 사용

#### 상가유형 전처리
- 상가유형에 따라 결측값을 확인하고 결측값 처리
- 상가유형에서의 공실여부를 1과 0 으로 이진 분류(공실 : 1, 비공실 : 0)

In [119]:
subway_rent.isnull().sum()

상가유형     0
역사명      0
면적      59
dtype: int64

In [121]:
# '면적'에서 결측값이 발견된 역사명 확인
subway_rent.loc[subway_rent["면적"].isnull(),"역사명"].unique()

array(['을지로입구역', '고속터미널역', '화곡역', '공덕역', '강남구청역', '반포역'], dtype=object)

In [123]:
# 결측값 삭제
subway_rent.dropna(inplace =True)

In [133]:
# 상가유형 공실 : 0 /비공실 : 1로 이진분류
subway_rent.loc[subway_rent['상가유형'] != '공실','상가유형'] = 0 # 비공실
subway_rent.loc[subway_rent['상가유형'] == '공실', '상가유형'] = 1 # 공실
subway_rent["상가유형"] = subway_rent["상가유형"].astype(int)

In [134]:
subway_rent

,상가유형,역사명,면적
0,0,서울역,33.00
1,0,시청역,29.73
2,0,시청역,57.60
3,0,시청역,25.00
4,0,시청역,25.00
...,...,...,...
1633,0,단대오거리역,54.03
1634,0,단대오거리역,75.09
1635,0,신흥역,40.00
1636,0,수진역,40.00


### 소상공인 상가정보
- 서울시 소상공인 정보를 활용해 역 주변 수급처에 대한 분석 진행
- 사용컬럼: '상권업종대분류명','상권업종중분류명','시군구명','행정동코드','행정동명','경도','위도'

In [138]:
store = pd.read_csv("C://Users//82102//TIL//Study//data//소상공인시장진흥공단_상가(상권)정보_서울_202106.csv", encoding = 'utf-8', usecols = ['상권업종대분류명','상권업종중분류명','시군구명','행정동코드','행정동명','경도','위도'])
store.head()

,상권업종대분류명,상권업종중분류명,시군구명,행정동코드,행정동명,경도,위도
0,음식,한식,서초구,1165062100,방배4동,126.991394,37.488375
1,음식,한식,동작구,1159063000,사당2동,126.980952,37.487105
2,음식,한식,종로구,1111061500,종로1.2.3.4가동,126.981794,37.572387
3,소매,가정/주방/인테리어,영등포구,1156062000,양평2동,126.897892,37.536700
4,음식,한식,서초구,1165053000,서초3동,127.009382,37.483436


#### 음식 및 농산물 소매관련 업종 선별
- 스마트팜에서 생산된 농작물들을 활용할 수 있는 "수급처"를 확인하기 위해 "상권업종대분류명", "상권업종중분류명"을 확인한다.

In [140]:
store["상권업종대분류명"].unique()

array(['음식', '소매', '학문/교육', '부동산', '관광/여가/오락', '생활서비스', '숙박', '스포츠'],
      dtype=object)

In [141]:
store["상권업종중분류명"].unique()

array(['한식', '가정/주방/인테리어', '의복의류', '건강/미용식품', '취미/오락관련소매', '학원-보습교습입시',
       '사진/광학/정밀기기소매', '유흥주점', '학원-창업취업취미', '자동차/자동차용품', '부동산중개',
       '도서관/독서실', '커피점/카페', '패스트푸드', '무도/유흥/가무', '주유소/충전소', '종합소매점',
       '닭/오리요리', '자동차/이륜차', 'PC/오락/당구/볼링등', '음/식료품소매', '세탁/가사서비스',
       '선물/팬시/기념품', '이/미용/건강', '사무/문구/컴퓨터', '화장품소매', '기타서비스업', '시계/귀금속소매',
       '가방/신발/액세서리', '가전제품소매', '운동/경기용품소매', '학원-예능취미체육', '학원기타', '일식/수산물',
       '물품기기대여', '제과제빵떡케익', '학문교육기타', '분식', '양식', '사진', '애견/애완/동물',
       '학원-음악미술무용', '대행업', '주택수리', '철물/난방/건설자재소매', '별식/퓨전요리', '모텔/여관/여인숙',
       '호텔/콘도', '의약/의료품소매', '학원-어학', '유아교육', '인력/고용/용역알선', '책/서적/도서',
       '기타음식업', '광고/인쇄', '가구소매', '유아용품', '개인/가정용품수리', '운송/배달/택배',
       '예식/의례/관혼상제', '요가/단전/마사지', '실내운동시설', '운영관리시설', '법무세무회계', '부페',
       '기타판매업', '연극/영화/극장', '개인서비스', '평가/개발/관리', '장례/묘지', '중식', '분양',
       '중고품소매/교환', '대중목욕탕/휴게', '예술품/골동품/수석/분재', '페인트/유리제품소매',
       '학원-자격/국가고시', '스포츠/운동', '음식배달서비스', '종교용품판매', '경마/경륜/성인오락',
       '행사/이벤트', '민박/하숙', '학원

In [142]:
store[store["상권업종대분류명"]== '소매']['상권업종중분류명'].unique()

array(['가정/주방/인테리어', '의복의류', '건강/미용식품', '취미/오락관련소매', '사진/광학/정밀기기소매',
       '자동차/자동차용품', '종합소매점', '음/식료품소매', '선물/팬시/기념품', '사무/문구/컴퓨터', '화장품소매',
       '시계/귀금속소매', '가방/신발/액세서리', '가전제품소매', '운동/경기용품소매', '애견/애완/동물',
       '철물/난방/건설자재소매', '의약/의료품소매', '책/서적/도서', '가구소매', '유아용품', '기타판매업',
       '중고품소매/교환', '예술품/골동품/수석/분재', '페인트/유리제품소매', '종교용품판매'], dtype=object)

In [144]:
store[store["상권업종대분류명"]=='음식']['상권업종중분류명'].unique()

array(['한식', '유흥주점', '커피점/카페', '패스트푸드', '닭/오리요리', '일식/수산물', '제과제빵떡케익',
       '분식', '양식', '별식/퓨전요리', '기타음식업', '부페', '중식', '음식배달서비스'],
      dtype=object)

In [ ]:
#### 상권업종 대분류명/중분류명을 확인한 결과 분석에 필요한 업종을 다음과 같이 선택한다
- 대분류 : '음식', '소매'
- 중분류 : '음식' 업종의 전체, '소매'업종의 '음식료품'